In [1]:
using ITensors
using JLD2

In [2]:
function DMRG(N; g)
    sites = siteinds("S=1/2",N)
    
    H = OpSum()
    for i in 1:N-1
        H += -1,"Sz",i,"Sz",i+1
    end
    for i in 1:N
        H += -g,"Sx",i
    end
    
    MPH = MPO(H,sites)
    ψ = randomMPS(sites,10)
    
    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,200,400,800)
    setcutoff!(sweeps, 1E-8)
    E_0, ψ_0 = dmrg(MPH, ψ, sweeps, outputlevel = 0)
    
    return ψ_0
end

DMRG (generic function with 1 method)

In [3]:
g = collect(0.0:0.01:1.0)
ψ = Vector{MPS}(undef, length(g))

for vals in g
    @inbounds i = Int(round(vals*100)) + 1
    ψ[i] = DMRG(80, g = vals)
end

In [4]:
os = OpSum()
for j=1:80
    os += 1/80,"Sz",j
end

M² = fill(0.0, length(ψ))
for i in 1:length(ψ)
    @inbounds sites = siteinds(ψ[i])
    M = MPO(os,sites)
    M_sq = apply(M,M)
    @inbounds M²[i] = inner(ψ[i]',M_sq,ψ[i])
end

In [5]:
save_object("LongRangeOrder.jld2", M²)